In [99]:
import numpy as np
import math

# 1. A basic Value wrapper for a scalar

In [8]:
class Value:
    def __init__(self, data: float, label: str = "") -> None:
        self.data =  data
        self.label = label
        self.grad = 0.0
        self._backward = lambda : None

    def __repr__(self) -> str:
        return f"Value(data = {self.data})"

    def __add__(self, other):
        return Value(data = self.data + other.data)

    def __mul__(self, other):
        return Value(data = self.data * other.data)

    def __rmul__(self, other):
        return self * other

In [9]:
a = Value(data=1.0, label='a')
b = Value(data=2.0, label='b')

a + b

Value(data = 3.0)

In [10]:
a * b

Value(data = 2.0)

In [11]:
b * a

Value(data = 2.0)

# 2. Define backward function
For each operation, we are creating a new Value object derived by the callee operation. That object must embed information for how to propagate gradient backward through it. This gradient is calculated by Chain rule.

In [84]:
class Value:
    def __init__(self, data: float, label: str = "") -> None:
        self.data =  data
        self.label = label
        self.grad = 0.0
        self._backward = lambda : None

    def __repr__(self) -> str:
        return f"Value(data = {self.data})"

    def __add__(self, other):
        # Forward pass
        other = Value(data=other) if not isinstance(other, Value) else other
        output = Value(data = self.data + other.data)

        # Backward pass function
        def _backward():
            self.grad += output.grad * 1.0   # We use += because gradients can be accumulated from many paths
            other.grad += output.grad * 1.0  # depending on how many nodes this self node connects to.
        output._backward = _backward
        
        return output

    def __mul__(self, other):
        # Forward pass
        output = Value(data = self.data * other.data)

        # Backward pass function
        def _backward():
            self.grad += output.grad * other.data
            other.grad += output.grad * self.data
        output._backward = _backward

        return output

    def tanh(self):
        # Forward pass
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        output = Value(data = t)
        
        # Backward pass function
        def _backward():
            self.grad += (1 - t**2) * output.grad
        output._backward = _backward

        return output
        
    def __rmul__(self, other):
        return self * other

In [108]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

In [111]:
print(f"""
Data:
    o: {o.data:.3f}
    n: {n.data:.3f}
    b: {b.data:.3f}
    x1w1x2w2: {x1w1x2w2.data:.3f}
    x1w1: {x1w1.data:.3f}
    x2w2: {x2w2.data:.3f}
    x1: {x1.data:.3f}
    w1: {w1.data:.3f}
    x2: {x2.data:.3f}
    w2: {w2.data:.3f}
""")


Data:
    o: 0.707
    n: 0.881
    b: 6.881
    x1w1x2w2: -6.000
    x1w1: -6.000
    x2w2: 0.000
    x1: 2.000
    w1: -3.000
    x2: 0.000
    w2: 1.000



In [109]:
"""
    To start up gradient backpropagation, the final node's gradient is set to one.
    e.g. the gradient of the loss function wrt. itself is one.
"""

o.grad = 1.0

"""
    We need call backward function for parent before children for every pair of them
    so that gradient flows backward in a correct order.
"""
o._backward()
n._backward()
b._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()

print(f"""
Gradient:
    o: {o.grad:.3f}
    n: {n.grad:.3f}
    b: {b.grad:.3f}
    x1w1x2w2: {x1w1x2w2.grad:.3f}
    x1w1: {x1w1.grad:.3f}
    x2w2: {x2w2.grad:.3f}
    x1: {x1.grad:.3f}
    w1: {w1.grad:.3f}
    x2: {x2.grad:.3f}
    w2: {w2.grad:.3f}
""")


Gradient:
    o: 1.000
    n: 0.500
    b: 0.500
    x1w1x2w2: 0.500
    x1w1: 0.500
    x2w2: 0.500
    x1: -1.500
    w1: 1.000
    x2: 0.500
    w2: 0.000



# 3. Convenient backward function
Calling backward funtion for each node manually is tedious. We need a function that triggers backward propagation from the end back to the intput nodes automatically.

To do so, we need a way to traverse through the computational graph in the right order. So, we need to keep additional information as to how each node is constructed (e.g. its previous nodes)

c = a + b <br>
c.previous = (a, b)

In [116]:
class Value:
    def __init__(self, data: float, label: str = "", previous = []) -> None:
        self.data =  data
        self.label = label
        self.grad = 0.0
        self._backward = lambda : None
        self._previous = previous

    def __repr__(self) -> str:
        return f"Value(data = {self.data})"

    def __add__(self, other):
        # Forward pass
        other = Value(data=other) if not isinstance(other, Value) else other
        output = Value(data = self.data + other.data, previous=[self, other])

        # Backward pass function
        def _backward():
            self.grad += output.grad * 1.0
            other.grad += output.grad * 1.0
        output._backward = _backward
        
        return output

    def __mul__(self, other):
        # Forward pass
        other = Value(data=other) if not isinstance(other, Value) else other
        output = Value(data = self.data * other.data, previous=[self, other])

        # Backward pass function
        def _backward():
            self.grad += output.grad * other.data
            other.grad += output.grad * self.data
        output._backward = _backward

        return output

    def tanh(self):
        # Forward pass
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        output = Value(data = t, previous=[self])
        
        # Backward pass function
        def _backward():
            self.grad += (1 - t**2) * output.grad
        output._backward = _backward

        return output
        
    def __rmul__(self, other):
        return self * other

In [117]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

We can use Breadth First Search to find the correct order.

In [129]:
def BFS(root):
    output_order = []
    stack = [root]

    while len(stack) > 0:
        top = stack.pop(0)
        output_order.append(top)

        if len(top._previous):
            stack.extend(top._previous)
        
    return output_order

print([e.label for e in BFS(o)])

['o', 'n', 'x1*w1 + x2*w2', 'b', 'x1*w1', 'x2*w2', 'x1', 'w1', 'x2', 'w2']


So, when we propagate gradient by this order, the gradient will flow correctly from output to input nodes.

In [125]:
class Value:
    def __init__(self, data: float, label: str = "", previous = []) -> None:
        self.data =  data
        self.label = label
        self.grad = 0.0
        self._backward = lambda : None
        self._previous = previous

    def __repr__(self) -> str:
        return f"Value(data = {self.data})"

    def __add__(self, other):
        # Forward pass
        other = Value(data=other) if not isinstance(other, Value) else other
        output = Value(data = self.data + other.data, previous=[self, other])

        # Backward pass function
        def _backward():
            self.grad += output.grad * 1.0
            other.grad += output.grad * 1.0
        output._backward = _backward
        
        return output

    def __mul__(self, other):
        # Forward pass
        other = Value(data=other) if not isinstance(other, Value) else other
        output = Value(data = self.data * other.data, previous=[self, other])

        # Backward pass function
        def _backward():
            self.grad += output.grad * other.data
            other.grad += output.grad * self.data
        output._backward = _backward

        return output

    def tanh(self):
        # Forward pass
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        output = Value(data = t, previous=[self])
        
        # Backward pass function
        def _backward():
            self.grad += (1 - t**2) * output.grad
        output._backward = _backward

        return output
        
    def __rmul__(self, other):
        return self * other

    def backward(self):
        self.grad = 1.0
        def BFS(root):
            output_order = []
            stack = [root]

            while len(stack) > 0:
                top = stack.pop(0)
                output_order.append(top)

                if len(top._previous):
                    stack.extend(top._previous)
                
            return output_order

        output_order = BFS(self)
        for node in output_order:
            node._backward()

In [126]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

o.backward()

print(f"""
Gradient:
    o: {o.grad:.3f}
    n: {n.grad:.3f}
    b: {b.grad:.3f}
    x1w1x2w2: {x1w1x2w2.grad:.3f}
    x1w1: {x1w1.grad:.3f}
    x2w2: {x2w2.grad:.3f}
    x1: {x1.grad:.3f}
    w1: {w1.grad:.3f}
    x2: {x2.grad:.3f}
    w2: {w2.grad:.3f}
""")


Gradient:
    o: 1.000
    n: 0.500
    b: 0.500
    x1w1x2w2: 0.500
    x1w1: 0.500
    x2w2: 0.500
    x1: -1.500
    w1: 1.000
    x2: 0.500
    w2: 0.000



Another example

In [127]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b    ; d.label = 'd'
e = a + b    ; e.label = 'e'
f = d * e    ; f.label = 'f'

f.backward()

In [128]:
print(f"""
Gradient:
    f: {f.grad:.3f}
    d: {d.grad:.3f}
    e: {e.grad:.3f}
    a: {a.grad:.3f}
    b: {b.grad:.3f}
""")


Gradient:
    f: 1.000
    d: 1.000
    e: -6.000
    a: -3.000
    b: -8.000

